### KoNLPy 를 활용한 한국어 분석

In [5]:
from konlpy.tag import Kkma
from konlpy.utils import pprint

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [61]:
kkma = Kkma()
pprint(kkma.sentences('네, 안녕하세요. 반갑습니다.'))

['네, 안녕하세요.', '반갑습니다.']


In [62]:
pprint(kkma.nouns('질문이나 건의사항은 깃헙 이슈 트래커에 남겨주세요.'))

['질문', '건의', '건의사항', '사항', '깃헙', '이슈', '트래커']


In [63]:
pprint(kkma.pos('오류보고는 실행환경, 에러메세지와함께 설명을 최대한 상세히!^^'))

[('오류', 'NNG'),
 ('보고', 'NNG'),
 ('는', 'JX'),
 ('실행', 'NNG'),
 ('환경', 'NNG'),
 (',', 'SP'),
 ('에러', 'NNG'),
 ('메세지', 'NNG'),
 ('와', 'JKM'),
 ('함께', 'MAG'),
 ('설명', 'NNG'),
 ('을', 'JKO'),
 ('최대한', 'NNG'),
 ('상세히', 'MAG'),
 ('!', 'SF'),
 ('^^', 'EMO')]


In [64]:
pprint(kkma.morphs('오류보고는 실행환경, 에러메세지와함께 설명을 최대한상세히!^^'))

['오류',
 '보고',
 '는',
 '실행',
 '환경',
 ',',
 '에러',
 '메세지',
 '와',
 '함께',
 '설명',
 '을',
 '최대한',
 '상세히',
 '!',
 '^^']


In [16]:
# p445
from konlpy.tag import Okt

In [17]:
okt_tag = Okt()
okt_tag.morphs('질문이나 건의사항은 깃헙 이슈 트래커에 남겨주세요.')

['질문', '이나', '건의', '사항', '은', '깃', '헙', '이슈', '트래커', '에', '남겨주세요', '.']

In [18]:
from sklearn.feature_extraction.text import CountVectorizer

In [19]:
cv = CountVectorizer(tokenizer=okt_tag.morphs) # 문장 > 벡터로 변환할때 단어 자르는 기능을 konlpy를 사용

In [20]:
sentences = ['안녕, 나는 남자야 남자!', '안녕, 나는 여자야 여자!']

In [23]:
cv.fit(sentences) # 토큰(단어)을 다 잘라서 어휘 사전을 준비한다.
cv.vocabulary_

{'안녕': 5, ',': 1, '나': 2, '는': 4, '남자': 3, '야': 6, '!': 0, '여자': 7}

In [22]:
sparse_matrix = cv.transform(sentences) # 문장을 희소 행렬로 변환

### 네이버 영화 리뷰

In [34]:
movie = pd.read_csv('ratings_train.txt', delimiter='\t')
movie.shape

(150000, 3)

In [35]:
movie.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [36]:
movie.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 3 columns):
id          150000 non-null int64
document    149995 non-null object
label       150000 non-null int64
dtypes: int64(2), object(1)
memory usage: 3.4+ MB


In [37]:
movie = movie.drop(columns='id').dropna()
movie.head()

,document,label
0,아 더빙.. 진짜 짜증나네요 목소리,0
1,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,너무재밓었다그래서보는것을추천한다,0
3,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [38]:
movie.isna().sum()

document    0
label       0
dtype: int64

In [39]:
# 속도를 위해 데이터를 좀 줄인다.
m = 1000
movie = movie.iloc[:m]

In [40]:
X = movie.document
y = movie.label

In [66]:
from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(tokenizer=Okt().morphs, ngram_range=(1, 2))

In [67]:
cv.fit(X)

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=<bound method Okt.morphs of <konlpy.tag._okt.Okt object at 0x000002A014C2FEB8>>,
        vocabulary=None)

In [68]:
len(cv.vocabulary_)

16350

In [69]:
X_sm = cv.transform(X)

In [70]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression, LogisticRegression #예측, 분류

In [71]:
scores = cross_val_score(LogisticRegression(), X_sm, y, cv=3)
scores, np.mean(scores)

C:\Users\wtime\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(array([0.73053892, 0.70870871, 0.6966967 ]), 0.7119814425203647)

In [72]:
model = LogisticRegression().fit(X_sm, y)

In [89]:
sentences = ['두시간 동안 잘 자고 나왔습니다.'
            , '이 영화 최고 인듯.'
            , '제길 죽을 것 같아! 악마 같은 중독 영화야! 개 좋음.'
            , '최고 최고 최고 최고로 돈 낭비']
model.predict(cv.transform(sentences))

array([0, 1, 0, 1], dtype=int64)

In [86]:
# 특정단어가 특정문서에서만 많이 나오는 경우 중요한 단어로 인식
# 즉, 문서내 단어의 빈도수를 중요하게 여기는 분석방법이다. 정확도가 조금 증가한다.
from sklearn.feature_extraction.text import TfidfVectorizer 
cv = TfidfVectorizer(tokenizer=Okt().morphs, ngram_range=(1, 2))

In [87]:
cv.fit(X)
X_sm = cv.transform(X)

In [88]:
scores = cross_val_score(LogisticRegression(), X_sm, y, cv=3)
scores, np.mean(scores)

C:\Users\wtime\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(array([0.71556886, 0.75375375, 0.69369369]), 0.7210054365742988)

In [104]:
url = 'https://movie.naver.com/movie/point/af/list.nhn?st=mcode&sword=173123&target=after&page=1'
df = pd.read_html(url)
len(df)

2

In [105]:
df[1]

,번호,평점,평점.1,140자평,글쓴이·날짜
0,15907453,NaN,7,스파이더맨: 파 프롬 홈 그림은 좋으나 내용이 고구마야 답답해 그나마 쿠키영상이 ...,chad****19.07.16
1,15907451,NaN,10,스파이더맨: 파 프롬 홈 전편보다 진화된 액션! 톰 홀랜드 너무 잘생김~ 신고,gpwl****19.07.16
2,15907440,NaN,10,스파이더맨: 파 프롬 홈 영화관에서 과도한 스킨쉽하지맙시다 아니 쌍..왜 영화관에...,tjgu****19.07.16
3,15907413,NaN,10,스파이더맨: 파 프롬 홈 반전 대박 입니,NaN


### 텍스트 다루기
- 한국어 형태분석
>from konlpy.tag import Okt
- 문장을 분석을 위한 클래스 지정
>from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer<BR>
>cv = CountVectorizer(tokenizer=Okt().morphs, ngram_range=(1, 2)) # ngram_range 는 여러 단어를 합치도록 지정가능 <BR>
>or<BR>
>cv = TfidfVectorizer(tokenizer=Okt().morphs, ngram_range=(1, 2)) # 빈도수를 활용
- Fitting(단어사전 작성) 및 단어 분석
> cv.fit(X) <BR>
> X_sm = cv.transform(X)
- \*분류에 대한 평가 : 추천/비추천의 두가지로 분류하는 문제이기에 분류로 처리한다.
>from sklearn.linear_model import LogisticRegression <BR>
>from sklearn.model_selection import cross_val_score <BR>
><BR>
>scores = cross_val_score(LogisticRegression(), X_sm, y, cv=3) <BR>
>scores, np.mean(scores)
- 분류모델 fitting 및 새로운 문장에 대한 분류 예측
>model = LogisticRegression().fit(X_sm, y) <BR>
><BR>
>sentences = ['예측하고 싶은 문장'] <BR>
>model.predict(cv.transform(sentences)) <BR>
- 기타 : `LinearRegression`는 예측에 사용하는 모델